### Json Parsing And Processing

In [1]:
import json
import os
os.makedirs("data/json_files",exist_ok=True)

In [2]:
# Sample nested JSON data
json_data = {
    "company": "TechCorp",
    "employees": [
        {
            "id": 1,
            "name": "John Doe",
            "role": "Software Engineer",
            "skills": ["Python", "JavaScript", "React"],
            "projects": [
                {"name": "RAG System", "status": "In Progress"},
                {"name": "Data Pipeline", "status": "Completed"}
            ]
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "role": "Data Scientist",
            "skills": ["Python", "Machine Learning", "SQL"],
            "projects": [
                {"name": "ML Model", "status": "In Progress"},
                {"name": "Analytics Dashboard", "status": "Planning"}
            ]
        }
    ],
    "departments": {
        "engineering": {
            "head": "Mike Johnson",
            "budget": 1000000,
            "team_size": 25
        },
        "data_science": {
            "head": "Sarah Williams",
            "budget": 750000,
            "team_size": 15
        }
    }
}

In [3]:
json_data

{'company': 'TechCorp',
 'employees': [{'id': 1,
   'name': 'John Doe',
   'role': 'Software Engineer',
   'skills': ['Python', 'JavaScript', 'React'],
   'projects': [{'name': 'RAG System', 'status': 'In Progress'},
    {'name': 'Data Pipeline', 'status': 'Completed'}]},
  {'id': 2,
   'name': 'Jane Smith',
   'role': 'Data Scientist',
   'skills': ['Python', 'Machine Learning', 'SQL'],
   'projects': [{'name': 'ML Model', 'status': 'In Progress'},
    {'name': 'Analytics Dashboard', 'status': 'Planning'}]}],
 'departments': {'engineering': {'head': 'Mike Johnson',
   'budget': 1000000,
   'team_size': 25},
  'data_science': {'head': 'Sarah Williams',
   'budget': 750000,
   'team_size': 15}}}

In [4]:
with open('data/json_files/company_data.json', 'w') as f:
    json.dump(json_data, f, indent=2)

In [5]:
# Save JSON Lines format
jsonl_data = [
    {"timestamp": "2024-01-01", "event": "user_login", "user_id": 123},
    {"timestamp": "2024-01-01", "event": "page_view", "user_id": 123, "page": "/home"},
    {"timestamp": "2024-01-01", "event": "purchase", "user_id": 123, "amount": 99.99}
]

with open('data/json_files/events.jsonl', 'w') as f:
    for item in jsonl_data:
        f.write(json.dumps(item) + '\n')

## Json Processing Stratergies

In [6]:
from langchain_community.document_loaders import JSONLoader
import json

## MEthod1 : JsonLoader With jq_schema
print("1️⃣ JSONLoader - Extract specific fields")

# Extract employee information
employee_loader = JSONLoader(
    file_path='data/json_files/company_data.json',
    jq_schema='.employees[]',  # jq query to extract each employee
    text_content=False  # Get full JSON objects
)

employee_docs = employee_loader.load()
print(f"Loaded {len(employee_docs)} employee documents")
print(f"First employee: {employee_docs[0].page_content[:200]}...")
print(employee_docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


1️⃣ JSONLoader - Extract specific fields
Loaded 2 employee documents
First employee: {"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status"...
[Document(metadata={'source': 'C:\\Data_Science\\RAG\\Project\\DataIngestParsing\\data\\json_files\\company_data.json', 'seq_num': 1}, page_content='{"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status": "Completed"}]}'), Document(metadata={'source': 'C:\\Data_Science\\RAG\\Project\\DataIngestParsing\\data\\json_files\\company_data.json', 'seq_num': 2}, page_content='{"id": 2, "name": "Jane Smith", "role": "Data Scientist", "skills": ["Python", "Machine Learning", "SQL"], "projects": [{"name": "ML Model", "status": "In Progress"}, {"name": "Analytics Dashboa

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
# Processing from the internet

In [13]:
import os
import requests
import json

# Try to import Document from common places — flexible for different langchain versions
try:
    from langchain_core.documents import Document
except Exception:
    try:
        from langsmith.schemas import Document
    except Exception:
        # Final fallback: create a simple Document-like dict if import fails
        Document = None

os.environ["USER_AGENT"] = "DemoScript/1.0"
HEADERS = {"User-Agent": os.environ["USER_AGENT"]}
URL = "https://jsonplaceholder.typicode.com/posts"

def fetch_and_build_documents(url: str):
    resp = requests.get(url, headers=HEADERS, timeout=15)
    resp.raise_for_status()
    data = resp.json()

    docs = []
    for i, item in enumerate(data):
        text = json.dumps(item, ensure_ascii=False)  # store JSON as readable text
        meta = {"source": url, "seq_num": i, "id": item.get("id")}
        if Document is None:
            # fallback: create simple dict-like document
            docs.append({"page_content": text, "metadata": meta})
        else:
            docs.append(Document(page_content=text, metadata=meta))
    return docs

if __name__ == "__main__":
    docs = fetch_and_build_documents(URL)
    print(type(docs[0]))
    print(f"Built {len(docs)} documents (manual).")
    # Printing preview safely in case the Document is a dict fallback
    preview = docs[0].page_content if hasattr(docs[0], "page_content") else docs[0]["page_content"]
    print(preview[:200], "...")
    print("Sample metadata:", (docs[0].metadata if hasattr(docs[0], "metadata") else docs[0]["metadata"]))


<class 'langchain_core.documents.base.Document'>
Built 100 documents (manual).
{"userId": 1, "id": 1, "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit", "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit ...
Sample metadata: {'source': 'https://jsonplaceholder.typicode.com/posts', 'seq_num': 0, 'id': 1}


In [14]:
# Method 2: Custom JSON processing for complex structures
from typing import List
from langchain_core.documents import Document
print("\n2️⃣ Custom JSON Processing")

def process_json_intelligently(filepath: str) -> List[Document]:
    """Process JSON with intelligent flattening and context preservation"""
    with open(filepath, 'r') as f:
        data = json.load(f)
    
    documents = []
    
    # Strategy 1: Create documents for each employee with full context
    for emp in data.get('employees', []):
        content = f"""Employee Profile:
        Name: {emp['name']}
        Role: {emp['role']}
        Skills: {', '.join(emp['skills'])}

        Projects:"""
        for proj in emp.get('projects', []):
            content += f"\n- {proj['name']} (Status: {proj['status']})"
        
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'data_type': 'employee_profile',
                'employee_id': emp['id'],
                'employee_name': emp['name'],
                'role': emp['role']
            }
        )
        documents.append(doc)

    return documents


2️⃣ Custom JSON Processing


In [15]:
process_json_intelligently("data/json_files/company_data.json")

[Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 1, 'employee_name': 'John Doe', 'role': 'Software Engineer'}, page_content='Employee Profile:\n        Name: John Doe\n        Role: Software Engineer\n        Skills: Python, JavaScript, React\n\n        Projects:\n- RAG System (Status: In Progress)\n- Data Pipeline (Status: Completed)'),
 Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 2, 'employee_name': 'Jane Smith', 'role': 'Data Scientist'}, page_content='Employee Profile:\n        Name: Jane Smith\n        Role: Data Scientist\n        Skills: Python, Machine Learning, SQL\n\n        Projects:\n- ML Model (Status: In Progress)\n- Analytics Dashboard (Status: Planning)')]